In [1]:
import torch
from sklearn import metrics

a = torch.sign(torch.randn(10000000)).cuda()
b = torch.sign(torch.randn(10000000)).cuda()


In [5]:
mean=5
var=10
a = torch.sign(torch.normal(mean,var,size=a.shape,device=a.device))
b = torch.sign(torch.normal(mean,var,size=b.shape,device=b.device))

In [3]:
import numpy as np
import math
def my_NMI(a,b):
    a_n = torch.zeros_like(a,device=a.device)
    a_n[torch.where(a==-1)]=2
    a_n[torch.where(a==0 )]=3
    a_n[torch.where(a==1 )]=5
    b_n = torch.zeros_like(b,device=b.device)
    b_n[torch.where(b==-1)]=11
    b_n[torch.where(b==0 )]=23
    b_n[torch.where(b==1 )]=31

    a_unis = torch.unique(a,return_counts=False).int().cpu().numpy()
    b_unis = torch.unique(b,return_counts=False).int().cpu().numpy()
    # print(a_unis,b_unis)
    # print(a_n)
    total = len(a)
    eps = 1.4e-45

    c_n_dict = {(-1, -1): 13, (-1, 0): 25, (-1, 1): 33, (0, -1): 14, (0, 0): 26, (0, 1): 34, (1, -1): 16, (1, 0): 28, (1, 1): 36}
    c_n = a_n + b_n
    
    # 计算mi
    mi=0
    for a_u in a_unis:
        for b_u in b_unis:
            key = (a_u,b_u)
            value = c_n_dict[key]

            px = 1.0*len(torch.where(a==a_u)[0])/total
            py = 1.0*len(torch.where(b==b_u)[0])/total
            pxy = 1.0*len(torch.where(c_n==value)[0])/total
            # print('key:{},pxy_idx:{}'.format(key,torch.where(c_n==value)[0]))
            mi = mi + pxy*math.log(pxy/(px*py)+eps,2)
    # print(mi)

    # 计算nmi
    hx = 0
    for a_u in a_unis:
        ida_occurCount = 1.0*len(torch.where(a==a_u)[0])
        hx = hx - (ida_occurCount/total)*math.log(ida_occurCount/total+eps,2)
    hy = 0
    for b_u in b_unis:
        idb_occurCount = 1.0*len(torch.where(b==b_u)[0])
        hy = hy - (idb_occurCount/total)*math.log(idb_occurCount/total+eps,2)
    nmi = 2.0*mi/(hx+hy)
    return nmi

In [6]:
my_NMI(a,b)

3.1340854732906822e-09

In [39]:
def NMI(A,B):
    #样本点数
    total = len(A)
    A_ids = set(A)
    B_ids = set(B)
    print(A_ids,B_ids)
    #互信息计算
    MI = 0
    eps = 1.4e-45
    for idA in A_ids:
        for idB in B_ids:
            idAOccur = np.where(A==idA)
            idBOccur = np.where(B==idB)
            idABOccur = np.intersect1d(idAOccur,idBOccur)
            px = 1.0*len(idAOccur[0])/total
            py = 1.0*len(idBOccur[0])/total
            pxy = 1.0*len(idABOccur)/total
            print(px,py,pxy)
            MI = MI + pxy*math.log(pxy/(px*py)+eps,2)
    # 标准化互信息
    Hx = 0
    for idA in A_ids:
        idAOccurCount = 1.0*len(np.where(A==idA)[0])
        Hx = Hx - (idAOccurCount/total)*math.log(idAOccurCount/total+eps,2)
    Hy = 0
    for idB in B_ids:
        idBOccurCount = 1.0*len(np.where(B==idB)[0])
        Hy = Hy - (idBOccurCount/total)*math.log(idBOccurCount/total+eps,2)
    MIhat = 2.0*MI/(Hx+Hy)
    return MIhat


In [3]:
#a=[-1,0,1]
c = [2,3,5] 
#b=[-1,0,1]
d = [11,23,31]
e = []
for i in c:
    f = []
    for j in d:
        f.append(i+j)
    print(f)



[13, 25, 33]
[14, 26, 34]
[16, 28, 36]


In [14]:
import numpy as np
import math
def NMI(a,b):
    a_n = torch.zeros_like(a,device=a.device)
    a_n[torch.where(a==-1)]=2
    a_n[torch.where(a==0 )]=3
    a_n[torch.where(a==1 )]=5
    b_n = torch.zeros_like(b,device=b.device)
    b_n[torch.where(b==-1)]=11
    b_n[torch.where(b==0 )]=23
    b_n[torch.where(b==1 )]=31

    total = len(a)
    eps = 1.4e-45

    c_n_dict = {13:(-1,-1),25:(-1,0),33:(-1,1),14:(0,-1),26:(0,0),34:(0,1),16:(1,-1),18:(1,0),36:(1,1)}
    c_n = (a_n + b_n).cpu().numpy()
    
    # 计算mi
    mi = 0
    for key,value in c_n_dict.items():
        ab_ = np.where(c_n==key)[0]
        if len(ab_) != 0:
            px = 1.0*len(torch.where(a==value[0])[0])/total
            py = 1.0*len(torch.where(b==value[1])[0])/total
            pxy = 1.0*len(ab_)/total
            mi = mi + pxy*math.log(pxy/(px*py)+eps,2)
    
    # 计算h
        

NMI(a,b)

[      0       1       3 ... 9999993 9999996 9999998]
[]
[      7       8      10 ... 9999988 9999992 9999999]
[]
[]
[]
[      2       4       5 ... 9999989 9999995 9999997]
[]
[      6       9      30 ... 9999983 9999991 9999994]


In [25]:
metrics.normalized_mutual_info_score(a,b)

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [5]:
import cupy as cp
from torch.utils.dlpack import to_dlpack,from_dlpack
from cupy import fromDlpack

In [7]:
A = fromDlpack(to_dlpack(a.cuda())) # pytorch -> cupy
B = fromDlpack(to_dlpack(b.cuda())) # pytorch -> cupy

/environment/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: This function is deprecated in favor of cupy.from_dlpack
  """Entry point for launching an IPython kernel.
/environment/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: This function is deprecated in favor of cupy.from_dlpack
  


In [16]:
a_n = a.numpy()
b_n = b.numpy()

In [22]:
import numpy as np
import math


def NMI(A,B):
    #样本点数
    total = len(A)
    A_ids = set(A)
    B_ids = set(B)
    print(A_ids,B_ids)
    #互信息计算
    MI = 0
    eps = 1.4e-45
    for idA in A_ids:
        for idB in B_ids:
            idAOccur = np.where(A==idA)
            idBOccur = np.where(B==idB)
            idABOccur = np.intersect1d(idAOccur,idBOccur)
            px = 1.0*len(idAOccur[0])/total
            py = 1.0*len(idBOccur[0])/total
            pxy = 1.0*len(idABOccur)/total
            print(px,py,pxy)
            MI = MI + pxy*math.log(pxy/(px*py)+eps,2)
    # 标准化互信息
    Hx = 0
    for idA in A_ids:
        idAOccurCount = 1.0*len(np.where(A==idA)[0])
        Hx = Hx - (idAOccurCount/total)*math.log(idAOccurCount/total+eps,2)
    Hy = 0
    for idB in B_ids:
        idBOccurCount = 1.0*len(np.where(B==idB)[0])
        Hy = Hy - (idBOccurCount/total)*math.log(idBOccurCount/total+eps,2)
    MIhat = 2.0*MI/(Hx+Hy)
    return MIhat


In [23]:
NMI(a_n,b_n)

{0.0, 1.0, -1.0} {1.0, -1.0}
4e-07 0.4999227 3e-07
4e-07 0.5000773 1e-07
0.499893 0.4999227 0.2498501
0.499893 0.5000773 0.2500429
0.5001066 0.4999227 0.2500723
0.5001066 0.5000773 0.2500343


1.1396913312833996e-07